In [ ]:
!pip3 install datasets

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import DistilBertTokenizer, DistilBertForTokenClassification
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, classification_report
import torch
import argparse
from os import path
import torch.nn as nn
import os
import random


In [ ]:
def get_fluent(s,label):
    new_s = []
    words = s.split(" ")
    for x,y in zip(words,label):
        if(y==0):
            new_s.append(x)

    return " ".join(new_s)

In [ ]:
task = "dc"
model_checkpoint = "google/muril-base-cased"
model_checkpoint = "xlm-roberta-base"
#model_checkpoint = "bert-base-multilingual-cased"

path_to_dataset = "./data/"

print("Model Name:", model_checkpoint)
experiment_id = "40PercentDataEXP2"
no_of_epochs = 40

batch_size = 16
label_list = ['is_fluent', 'is_disfluent'] # 0 -> isFluent , 1 -> isDisfluent
splits = [70,15,15]

test_sentences = []
gt_fluent_sentences = []

def get_dataset(path):

    with open(f"{path}/data40.dis", 'r') as dis_x, open(f"{path}/data40.labels", 'r') as labels_x:

        train_dict = {'labels': [], 'disfluent': []}
        valid_dict = {'labels': [], 'disfluent': []}
        test_dict  = {'labels': [], 'disfluent': []}

        disfluent_lines = dis_x.readlines()
        disfluent_labels = labels_x.readlines()
        temp = list(zip(disfluent_lines, disfluent_labels))
        random.shuffle(temp)
        res1, res2 = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        disfluent_lines, disfluent_labels = list(res1), list(res2)
        total_size = len(disfluent_lines)

        for i,(disfluent,labels) in enumerate(zip(disfluent_lines, disfluent_labels)):

            disfluent = disfluent.strip().split()
            labels = list(map(int, labels.strip().split()))

            if i < round(splits[0] * total_size / 100) :
                train_dict['disfluent'].append(disfluent)
                train_dict['labels'].append(labels)
            elif i < round(sum(splits[:2]) * total_size / 100):
                valid_dict['disfluent'].append(disfluent)
                valid_dict['labels'].append(labels)
            elif i < round(sum(splits) * total_size / 100):
                test_dict['disfluent'].append(disfluent)
                test_dict['labels'].append(labels)
                s = " ".join(disfluent)
                test_sentences.append(" ".join(disfluent))
                gt_fluent_sentences.append(get_fluent(s,labels))


    train_dataset = Dataset.from_dict(train_dict)
    valid_dataset = Dataset.from_dict(valid_dict)
    test_dataset = Dataset.from_dict(test_dict)

    return DatasetDict({'train': train_dataset, 'valid': valid_dataset, 'test': test_dataset})


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["disfluent"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


datasets = get_dataset(path="/content/drive/MyDrive/4thProject/data/40percentData")
print(datasets)

Model Name: xlm-roberta-base
DatasetDict({
    train: Dataset({
        features: ['labels', 'disfluent'],
        num_rows: 1113
    })
    valid: Dataset({
        features: ['labels', 'disfluent'],
        num_rows: 239
    })
    test: Dataset({
        features: ['labels', 'disfluent'],
        num_rows: 238
    })
})


In [ ]:
print("Data Size = "+str(len(datasets['train'])+len(datasets['valid'])+len(datasets['test'])))

Data Size = 1590


In [ ]:
" ".join(datasets['test'][0]['disfluent'])

'add the tire shop to umm my business contacts on my phone'

In [ ]:
test_sentences[:10]

['add the tire shop to umm my business contacts on my phone',
 'can you add sara to um skype',
 'i would like to add 7863434 as roberts home uh yea home number',
 'the insect life at high elevations is also remarkable for the pronounced predominance of wingless and flightless forms',
 'i want to log on myfitnesspal that i want to log a 3 hour hike starting at 11am',
 'add um dominos pizza to my contacts',
 'he has closed his eyes',
 'get my uh stock prices',
 'log my log the chicken pot pie i had for lunch',
 'successive u s administrations have ignored the fact that india must see her problems and her relationships in a different perspective']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
label_all_tokens = True
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model_name = model_checkpoint.split("/")[-1]




Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [ ]:
tokenized_datasets['train']

Dataset({
    features: ['labels', 'disfluent', 'input_ids', 'attention_mask'],
    num_rows: 1113
})

In [ ]:
tokenized_datasets['train']['disfluent'][:5]

[['um',
  'lets',
  'um',
  'lets',
  'post',
  'to',
  'facebook',
  'my',
  'dog',
  'is',
  'the',
  'cutest'],
 ['strava', 'log', 'that', 'i', 'did', 'umm', '25', 'squats'],
 ['lets', 'make', 'a', 'um', 'make', 'a', 'new', 'facebook', 'post'],
 ['please', 'log', 'my', 'uh', 'workout'],
 ['log', 'the', 'workout', 'for', 'me', 'for', 'me']]

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # 1-d prediction & true label
    true_predictions = [
        p for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label) if l != -100
    ]
    true_labels = [
        l for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label) if l != -100
    ]

    results = precision_recall_fscore_support(true_labels, true_predictions, zero_division=0)
    return {
        'accuracy': accuracy_score(true_labels, true_predictions),
        'precision0': torch.tensor(results[0])[0],
        'precision1': torch.tensor(results[0])[1],
        'recall0': torch.tensor(results[1])[0],
        'recall1': torch.tensor(results[1])[1],
        'f1score0': torch.tensor(results[2])[0],
        'f1score1': torch.tensor(results[2])[1],
    }


In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer
import torch

args = TrainingArguments(
    f"/content/drive/MyDrive/4thProject/data/40percentData/checkpoints/{experiment_id}",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=no_of_epochs,#10,
    weight_decay=0.01,
    eval_steps=100, #1000,
    logging_steps=100, #1000,
    save_steps=100, #1000,
    save_total_limit=1,
    load_best_model_at_end=True
)



data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train(resume_from_checkpoint="/content/drive/MyDrive/4thProject/data/40percentData/checkpoints/40PercentDataEXP1/checkpoint-1000")
#trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision0,Precision1,Recall0,Recall1,F1score0,F1score1
2000,0.005200,0.018821,0.996883,0.997998,0.989744,0.998397,0.987212,0.998197,0.988476


TrainOutput(global_step=2800, training_loss=0.002205039347921099, metrics={'train_runtime': 17188.7349, 'train_samples_per_second': 2.59, 'train_steps_per_second': 0.163, 'total_flos': 712786421316900.0, 'train_loss': 0.002205039347921099, 'epoch': 40.0})

In [ ]:
# Evaluate on test sentences
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# 1-d prediction & true label
true_predictions = [
    p for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label) if l != -100
]
true_labels = [
    l for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label) if l != -100
]

results = precision_recall_fscore_support(true_labels, true_predictions, zero_division=0)
print({
    'precision': results[0],
    'recall': results[1],
    'f1score': results[2]
})
print("Confusion Matrix:")
print(confusion_matrix(true_labels, true_predictions, normalize='all'))
print(classification_report(true_labels, true_predictions, target_names=label_list, zero_division=0))


{'precision': array([0.99433798, 0.98356164]), 'recall': array([0.99737877, 0.96505376]), 'f1score': array([0.99585605, 0.97421981])}
Confusion Matrix:
[[0.85794814 0.00225479]
 [0.00488538 0.13491169]]
              precision    recall  f1-score   support

   is_fluent       0.99      1.00      1.00      2289
is_disfluent       0.98      0.97      0.97       372

    accuracy                           0.99      2661
   macro avg       0.99      0.98      0.99      2661
weighted avg       0.99      0.99      0.99      2661



In [ ]:
confusion_matrix(true_labels, true_predictions, normalize='all')

array([[0.85794814, 0.00225479],
       [0.00488538, 0.13491169]])

In [ ]:
results[0]

array([0.99433798, 0.98356164])